# Requirements

1. Creating the Anaconda environement /tool_keaml/src/examples/keyword_extraction_environment.yml
2. After selecting the right Kernel for the jupyter notebook based on the created anaconda env, it should be possible to run the following codes


In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from sentence_transformers import SentenceTransformer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import sys
sys.path.append("/home/rtue/playground/jupyter-notebook/workspace/codes")
sys.path.append("/home/rtue/playground/jupyter-notebook/")
sys.path.append("/home/rtue/playground/python_projects/tool_keaml/src/")
sys.path.append("/home/rtue/playground/python_projects/tool_keaml/src/utils")
sys.path.append("/home/rtue/playground/jupyter-notebook/keyword_extraction")
import util
from training_data.data_prepare import DataPrepare
import prepare_input_for_YouTubeDNN as prepare_input_for_dnn
import youTube_pipeline
import encoding_pipeline

from bs4 import BeautifulSoup
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from termcolor import colored
from itertools import chain
from tqdm import tqdm
import re, nltk
import tensorflow as tf
import pandas as pd
import numpy as np
import random, time, ast, collections, pickle, faiss
from nltk.tokenize import word_tokenize
import itertools, time

from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from tensorflow.python.keras import backend as K
from deepmatch.models import YoutubeDNN
from tensorflow.python.keras.layers import Embedding, Lambda
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras import initializers
from deepctr import feature_column as fc_lib
from deepmatch.utils import sampledsoftmaxloss
from deepmatch.utils import recall_N

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepctr.feature_column import build_input_features
from deepctr.layers import DNN
from deepctr.layers.utils import NoMask, combined_dnn_input
from tensorflow.python.keras.models import Model
from deepmatch.layers import PoolingLayer
from deepmatch.utils import get_item_embedding
from deepmatch.inputs import input_from_feature_columns, create_embedding_matrix
from deepmatch.layers.core import SampledSoftmaxLayer, EmbeddingIndex

In [5]:
model = SentenceTransformer("AI-Growth-Lab/PatentSBERTa")
embedding_dim=768
max_len_token=model.get_max_seq_length()

In [6]:
path_dw_id_kw='/home/rtue/playground/python_projects/resources/df_np_id_kw.csv'
path_dw_id_tit_abst_claim ='/home/rtue/playground/python_projects/resources/df_np_id_tit_abst_claim_detden.csv'

lst_columns=['TIEN', 'ABEN', 'CLMEN']
lst_columns_all=["PPID","TIEN", "ABEN", "CLMEN"]

In [7]:
data=DataPrepare()

data.set_der_abst()
data.set_full_text()

data_id_kw, data_id_tit_abs_clm, data_kw_or=util.read_dataset(path_dw_id_kw, path_dw_id_tit_abst_claim)

dict_or_occ_kw=util.get_occurance_dict(data_kw_or,['KEYWORD'])

print(len(data.df_full_text))
print(len(data.dict_full_text))
print(len(data.df_der_text))
print(len(data.dict_der_text))
#data.df_full_text.head()

DataPrepare object has the following properties:
 df_full_text, dict_full_text, df_der_text, dict_der_text
rows_with_NaN: Empty DataFrame
Columns: [PPID, KEYWORD]
Index: []
rows_with_NaN: Empty DataFrame
Columns: [PPID, TIEN, ABEN, CLMEN, DETDEN]
Index: []
data_kw is merged version and len(data_kw): 271137
occurance dictionary len: 79450
15249
15239
25534
14244


In [8]:
#FILTER DATA (prune the keywords)

list_ele_1=util.get_list_ele_len_less_than(dict_or_occ_kw)
list_ele_3 = util.get_list_pha_len_less(dict_or_occ_kw)
list_ele_4 = util.get_list_ele_occ_more_than(dict_or_occ_kw)

#set parameters
n_less=100#, 70, 50]
lst_n_sample=[70]#[10, 20, 50, 70]
lst_n_epoch=[23]#23,30]
    
list_ele_2=util.get_list_ele_occ_less_than(dict_or_occ_kw, n_less)

lst_all=set(list_ele_1 + list_ele_2+list_ele_3+list_ele_4)

print('original len df:', len(data_kw_or))
data_kw=util.remove_rows_contain_kw_in_list( lst_all,data_kw_or)
print('after removing len df:', len(data_kw))
dict_occ_kw_clean=util.get_occurance_dict(data_kw,['KEYWORD'])

original len df: 271137
removing rows from the df
after removing len df: 14957
occurance dictionary len: 77


In [9]:
#encode the data
data_kw_copy, embedding_matrix_tien, embedding_matrix_abst, embedding_matrix_clm= encoding_pipeline.generate_embeddings(data_kw.copy(), data, model)

data_kw_copy, lbe_ppid, lbe_kw = encode_with_label_encoder(data_kw_copy)

embedding_matrix_kw=encoding_pipeline.generate_kw_emb(lbe_kw, model)

print(np.isnan(embedding_matrix_tien).any())
print(np.isnan(embedding_matrix_abst).any())
print(np.isnan(embedding_matrix_clm).any())
print(len(data_kw_copy['KEYWORD']))

number of unique KWs: 77
False
False
False
14957


In [10]:
train_set, test_set = gen_train_test_split_and_generate_test_data(data_kw_copy, lst_columns,  m_kws=11, negsample=0)#MAKE IT 11

train_model_input, train_label = gen_model_input(train_set)
test_model_input, test_label = gen_model_input(test_set)


print('*******************************')
print('train_set len:', len(train_set))
print('test_set len:', len(test_set))

train_model_input, train_label = gen_model_input(train_set)
test_model_input, test_label = gen_model_input(test_set)

print('train_model_input len:', len(train_model_input))
print('test_model_input len:', len(test_model_input))

print('*******************************')

100%|██████████| 5356/5356 [00:00<00:00, 6404.94it/s]

len(train_set[0]): 6 len(test_set[0]): 6
len(train_set): 13586 len(test_set): 1371
*******************************
train_set len: 13586
test_set len: 1371
train_model_input len: 5
test_model_input len: 5
*******************************


In [11]:
train_set_dict=dict()
for e in train_set:
    id_=e[0]
    label=e[1]
    
    #print(e)
    if id_ in train_set_dict:
        train_set_dict[id_]+=[label]
    else:
        train_set_dict[id_]=[label]
        
test_set_dict=dict()
for e in test_set:
    id_=e[0]
    label=e[1]
    
    #print(e)
    if id_ in test_set_dict:
        test_set_dict[id_]+=[label]
    else:
        test_set_dict[id_]=[label]

In [12]:
doc_profile = data_kw_copy[lst_columns_all].drop_duplicates('PPID')
kw_profile = data_kw_copy[["KEYWORD"]].drop_duplicates('KEYWORD')

doc_profile.set_index("PPID", inplace=True)

In [13]:
for i in range(0,1):

    for n_sample in lst_n_sample:

        for n_epoch in lst_n_epoch:

            print('**************START*********************')
            print('th_for filtering kws:',n_less, ' n_neg samples:', n_sample, ' epoch_n:',  n_epoch)

            user_embs, item_embs=youTube_pipeline.train_model(data_kw_copy,train_model_input, train_label, test_model_input, 
                                                                embedding_matrix_tien, embedding_matrix_abst, embedding_matrix_clm,
                                                                n_sample, n_epoch )
            
            
            n_pred_=20
            p, r, f =youTube_pipeline.recall_N_after_post_processing(test_set_dict, kw_profile, data, data_id_tit_abs_clm, lbe_ppid,lbe_kw, user_embs, item_embs,  n_total=len(dict_occ_kw_clean),n_pred_=20)

            print(i,'th_for filtering kws:',n_less, ' n_neg samples:', n_sample, ' epoch_n:',  n_epoch, 'n_pred_:', n_pred_)
            print('**************FINISHED******************************')

**************START*********************
th_for filtering kws: 100  n_neg samples: 70  epoch_n: 23
I am in train_model function
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
0it [00:00, ?it/s]

I am in recall_N_after_post_processing


100it [00:01, 76.32it/s]


hit 1114
recall 0.8297619276692807
precision 0.5911065271587718
f_score: 0.6903914148653713
len rows 100
saved the dataframe
0 th_for filtering kws: 100  n_neg samples: 70  epoch_n: 23 n_pred_: 20
**************FINISHED******************************


In [3]:
def gen_train_test_split_and_generate_test_data(data_kw, columns_feature, m_kws=10, negsample=0):

    item_ids = data_kw['PPID'].unique()
    train_set = []
    test_set = []
    test_set_n_kws=m_kws
    
    
    data_kw_suffle=data_kw.sample(frac=1)
    data_kw_suffle.head()
    
    for index, (reviewerID, hist) in enumerate (tqdm(data_kw_suffle.groupby('PPID'))):

        pos_list = hist['KEYWORD'].tolist()
        tit='', 
        abst='', 
        clm='', 
        detden=''
        
        if 'TIEN' in columns_feature:
            tit= list(hist['TIEN'])[0]
        if 'ABEN' in columns_feature:
            abst= list(hist['ABEN'])[0]
        if 'CLMEN' in columns_feature:    
            clm= list(hist['CLMEN'])[0]
        if 'DETDEN' in columns_feature:    
            detden= list(hist['DETDEN'])[0]

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
            
        set_test_ids=set([item[0] for item in test_set])
        
        if len(set_test_ids)<100 and len(pos_list)>=test_set_n_kws:
            for i in range(0, len(pos_list)):
                test_set.append((reviewerID, pos_list[i], 1,tit ,abst, clm))
        else:
            for i in range(0, len(pos_list)):
                train_set.append((reviewerID, pos_list[i], 1,tit ,abst, clm))
                
    random.shuffle(train_set)
    random.shuffle(test_set)

    print('len(train_set[0]):',len(train_set[0]),'len(test_set[0]):',len(test_set[0]))
    print('len(train_set):',len(train_set), 'len(test_set):',len(test_set))

    return train_set,test_set

In [2]:
def gen_model_input(train_set):
    
    train_uid = np.array([line[0] for line in train_set])
    train_iid = np.array([line[1] for line in train_set])#movie_id
    train_label = np.array([line[2] for line in train_set])
    
    train_tien_seq =np.array( [line[3] for line in train_set]) #title
    
    train_abst_seq = np.array([line[4] for line in train_set])#abst

    train_clm_seq = np.array([line[5] for line in train_set])#clm


    train_model_input = {"PPID": train_uid,"KEYWORD": train_iid,  
                         "tien":train_tien_seq,
                         "aben":train_abst_seq,
                        "clmen":train_clm_seq}
    
    return train_model_input, train_label


def encode_with_label_encoder(df):
    data_kw_copy=df.copy()
    
    features = ['PPID', 'KEYWORD']
    feature_max_idx = {}

    lbe_ppid = LabelEncoder()
    lbe_kw = LabelEncoder()

    data_kw_copy['PPID'] = lbe_ppid.fit_transform(data_kw_copy['PPID']) 
    data_kw_copy['KEYWORD'] = lbe_kw.fit_transform(data_kw_copy['KEYWORD'])

    print('number of unique KWs:', colored(data_kw_copy['KEYWORD'].max()+1, 'green'))
    
    return data_kw_copy, lbe_ppid, lbe_kw





def decode_dict_items(dict_t):
    dict_res= dict()
    
    for i, (uid, gt) in enumerate(test_set_dict.items()):
        try:
            id_real=lbe_ppid.inverse_transform([uid])[0]
            set_gt_decoded=set()

            for kw in gt:
                kw_str=lbe_kw.inverse_transform([kw])[0]
                set_gt_decoded.add(kw_str)
            
            dict_res[id_real]=set_gt_decoded
        except:
            print('exception in:',i)
            sys.exit()  
    
    return dict_res